# Dataset Preparation

## Let's now explore the stock infos

In [18]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 

In [19]:
missing_values = ["n/a", "na", "--"]

historical_stocks = pd.read_csv('/Users/elisacatena/Desktop/historical_stocks/historical_stocks.csv', na_values = missing_values)

Let's start with a small preview of the dataframes.

In [20]:
historical_stocks.head()

,ticker,exchange,name,sector,industry
0,PIH,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
1,PIHPP,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
2,TURN,NASDAQ,180 DEGREE CAPITAL CORP.,FINANCE,FINANCE/INVESTORS SERVICES
3,FLWS,NASDAQ,"1-800 FLOWERS.COM, INC.",CONSUMER SERVICES,OTHER SPECIALTY STORES
4,FCCY,NASDAQ,1ST CONSTITUTION BANCORP (NJ),FINANCE,SAVINGS INSTITUTIONS


In [21]:
historical_stocks.describe()

,ticker,exchange,name,sector,industry
count,6460,6460,6460,5020,5020
unique,6460,2,5462,13,136
top,PIH,NASDAQ,BANK OF AMERICA CORPORATION,FINANCE,MAJOR PHARMACEUTICALS
freq,1,3308,16,1022,419


We realize that there are 5 columns in the historical_stocks dataset:

- 'ticker' corresponds to the name of the share
- 'exchange' corresponds to the type of exchange made 
- 'name' refers the company's name
- 'sector' refers to the actual sector where the given company operates
- 'industry' specifies the type of services that can be provided


### Missing values treatment
We want all rows that present null values.

In [22]:
null_data = historical_stocks[historical_stocks.isnull().any(axis=1)]
null_data

,ticker,exchange,name,sector,industry
19,ABP,NASDAQ,ABPRO CORPORATION,NaN,NaN
42,SQZZ,NASDAQ,ACTIVE ALTS CONTRARIAN ETF,NaN,NaN
62,ACT,NASDAQ,ADVISORSHARES VICE ETF,NaN,NaN
100,ABDC,NASDAQ,ALCENTRA CAPITAL CORP.,NaN,NaN
124,SMCP,NASDAQ,ALPHAMARK ACTIVELY MANAGED SMALL CAP ETF,NaN,NaN
...,...,...,...,...,...
6393,WIA,NYSE,WESTERN ASSET/CLAYMORE U.S. TREASURY INFLATION...,NaN,NaN
6431,XFLT,NYSE,XAI OCTAGON FLOATING RATE & ALTERNATIVE INCOME...,NaN,NaN
6451,ZB^A,NYSE,ZIONS BANCORPORATION,NaN,NaN
6452,ZB^G,NYSE,ZIONS BANCORPORATION,NaN,NaN


We realize that, by standard, all rows that do not have sector, do not have industry either and vice-versa.

#### The first step is to identify if the companies switched their share name; in case they did, they can then contain the sector and industry present on another row

Right now we have the following missing values:

1. ticker         0
2. exchange       0
3. name           0
4. sector      1440
5. industry    1440

### This function checks for companies that changed their ticker name

If any did, we check if there are some other instances of that same company where the sector and industry information is present.

In [23]:
pd.options.mode.chained_assignment = None

# Extract all unique values from the 'name' column
names = null_data['name'].unique()

for companie in names:
    
    # Extract the subset of data whose value in the 'name' column matches the company name currently considered in the loop.
    data = historical_stocks[historical_stocks['name'] == companie]
    
    for index,row in data.iterrows():
        
        # If the value in the 'sector' column of the currently considered row is not null
        if(not pd.isnull(row['sector'])):
            
            # Save the values in the 'sector' and 'industry' column 
            sector = row['sector']
            industry = row['industry']
            
            # Replace the missing values in the 'industry' and 'sector' column
            tmp = historical_stocks[historical_stocks['name'] == row['name']]
            tmp["sector"] = tmp["sector"].fillna(sector)
            tmp['industry'] = tmp['industry'].fillna(industry)
            historical_stocks[historical_stocks['name'] == row['name']] = tmp


In [24]:
historical_stocks.isnull().sum()

ticker         0
exchange       0
name           0
sector      1018
industry    1018
dtype: int64

After this operation, we have different numbers of missing values, respectively:

1. ticker         0
2. exchange       0
3. name           0
4. sector      1018
5. industry    1018
6. dtype: int64

### In this part, all remaining missing values will be removed from the dataset, mainly for the reason that there is not sufficient information that allows us to fill these values, given the variety of sector and industries existant.

In [25]:
bad_tickers = historical_stocks[historical_stocks.isnull().any(axis=1)]


historical_stocks = historical_stocks.dropna(how='any',axis=0) 
historical_stocks.isnull().sum()

ticker      0
exchange    0
name        0
sector      0
industry    0
dtype: int64

In [26]:
bad_tickers

,ticker,exchange,name,sector,industry
19,ABP,NASDAQ,ABPRO CORPORATION,NaN,NaN
42,SQZZ,NASDAQ,ACTIVE ALTS CONTRARIAN ETF,NaN,NaN
62,ACT,NASDAQ,ADVISORSHARES VICE ETF,NaN,NaN
100,ABDC,NASDAQ,ALCENTRA CAPITAL CORP.,NaN,NaN
124,SMCP,NASDAQ,ALPHAMARK ACTIVELY MANAGED SMALL CAP ETF,NaN,NaN
...,...,...,...,...,...
6390,MNP,NYSE,"WESTERN ASSET MUNICIPAL PARTNERS FUND, INC.",NaN,NaN
6391,GFY,NYSE,WESTERN ASSET VARIABLE RATE STRATEGIC FUND INC.,NaN,NaN
6392,WIW,NYSE,WESTERN ASSET/CLAYMORE U.S TREASURY INFLATION ...,NaN,NaN
6393,WIA,NYSE,WESTERN ASSET/CLAYMORE U.S. TREASURY INFLATION...,NaN,NaN


In [27]:
bad_tickers["ticker"].unique().size

1018

Now we only have 5442 tickers

In [28]:
historical_stocks['ticker'].unique().size

5442

In [29]:
historical_stocks.to_csv("/Users/elisacatena/Desktop/historical_stocks.csv", index=False)

## Let's now explore the stock prices

In [30]:
historical_stock_prices = pd.read_csv('/Users/elisacatena/Desktop/historical_stocks/historical_stock_prices.csv', na_values = missing_values)

In [31]:
historical_stock_prices.head()

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14


In [32]:
historical_stock_prices.describe()

,open,close,adj_close,low,high,volume
count,2.097389e+07,2.097389e+07,2.097389e+07,2.097389e+07,2.097389e+07,2.097389e+07
mean,7.605823e+01,7.611403e+01,1.481184e+14,7.422064e+01,7.803857e+01,1.227043e+06
std,2.849639e+03,2.870159e+03,4.574674e+16,2.746059e+03,2.997937e+03,1.316686e+07
min,4.000000e-04,2.000000e-04,2.282650e-09,1.000000e-04,4.000000e-04,1.000000e+00
25%,7.500000e+00,7.500000e+00,4.620000e+00,7.360000e+00,7.630000e+00,2.210000e+04
50%,1.545000e+01,1.545000e+01,1.138199e+01,1.524000e+01,1.566000e+01,1.260000e+05
75%,2.972000e+01,2.972000e+01,2.472046e+01,2.928000e+01,3.010000e+01,6.074000e+05
max,2.034000e+06,1.779750e+06,1.894962e+19,1.440000e+06,2.070000e+06,4.483504e+09


We realize that there are 5 columns in the historical_stock_prices dataset:

- 'ticker' corresponds to the name of the share
- 'open' describe the open price of that share in a specific day
- 'close' describe the final share price in the end of a day
- 'adj-close' it´s a tricky column, describes the ajudsted price of a share, thats normally different from the close price
- 'low' is the lowest value paid for that share
- 'high' is the highest value paid for that share
- 'volume' of shares purchased in that day
- 'date' represents the date (year-month-day)

### Missing values treatment
We want all rows that present null values.

In [33]:
historical_stock_prices.isnull().sum()

ticker       0
open         0
close        0
adj_close    0
low          0
high         0
volume       0
date         0
dtype: int64

In [34]:
null_data = historical_stock_prices[historical_stock_prices.isnull().any(axis=1)]
null_data

,ticker,open,close,adj_close,low,high,volume,date


As we can notice, this dataset doesn't contain any missing value

In [35]:
historical_stock_prices["ticker"].unique().size

5685

In this dataset we have 5685 different tickers, in the previous dataset we had 5441 tickers, so we have to eliminate some tickers here.

In [36]:
pd.options.mode.chained_assignment = None
tickers = bad_tickers['ticker'].unique()

ind = []

for index, row in historical_stock_prices.iterrows():
    if(row['ticker'] in tickers):
        ind.append(index)
        
ind = np.asarray(ind)
new_historical_stock_prices = historical_stock_prices.drop(ind)

In [37]:
new_historical_stock_prices["ticker"].unique().size

4774

In [38]:
new_historical_stock_prices.to_csv("/Users/elisacatena/Desktop/historical_stock_prices.csv", index=False)

In [39]:
merged_data = pd.merge(new_historical_stock_prices, historical_stocks, on='ticker')


merged_data.to_csv("/Users/elisacatena/Desktop/merged_data.csv", index=False)

merged_data["ticker"].unique().size

4774

In [40]:
merged_data.isnull().sum()

ticker       0
open         0
close        0
adj_close    0
low          0
high         0
volume       0
date         0
exchange     0
name         0
sector       0
industry     0
dtype: int64